# 🚀 **Tiny GPT(GPT 간소화 버전) 만들기**



---



- **요약**
    - **출처** : book `만들면서 배우는 생성 AI' 한빛미디어 Chapter9 트랜스포머
    - 훈련데이터셋 : **와인 리뷰 데이터셋**, https://www.kaggle.com/datasets/boh010/winemagdata130kv2json?select=winemag-data-130k-v2.json
    - [**Keras GPT 튜토리얼**] 참고 : https://keras.io/examples/generative/text_generation_with_miniature_gpt/

---


In [1]:
import numpy as np
import json
import re
import string
from IPython.display import display, HTML

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

## **0. 파라미터 설정** <a name="parameters"></a>

In [2]:
VOCAB_SIZE = 10000      # 사용할 단어 사전의 크기 설정
MAX_LEN = 80            # 문장의 최대 길이 설정
EMBEDDING_DIM = 256     # 임베딩 벡터의 차원 설정
KEY_DIM = 256           # 어텐션 메커니즘의 키 차원 설정
N_HEADS = 2             # 멀티 헤드 어텐션에서 사용할 헤드 개수 설정
FEED_FORWARD_DIM = 256  # 트랜스포머 블록 내 피드 포워드 네트워크의 은닉층 차원 설정
VALIDATION_SPLIT = 0.2  # 훈련 데이터셋에서 검증 데이터셋으로 분할할 비율 설정
SEED = 42               # 랜덤 시드 설정 (재현성을 위해)
LOAD_MODEL = False      # 미리 학습된 모델을 로드할지 여부 설정
BATCH_SIZE = 32         # 훈련 시 사용할 배치 크기 설정
EPOCHS = 5              # 훈련 에포크(epoch) 수 설정

## **1. 데이터 로드** <a name="load"></a>

In [3]:
import sys

# 코랩일 경우 노트북에서 celeba 데이터셋을 받습니다.
if 'google.colab' in sys.modules:
    # 캐글-->Setttings-->API-->Create New Token에서
    # kaggle.json 파일을 만들어 코랩에 업로드하세요.
    from google.colab import files
    files.upload()
    !mkdir ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    # celeba 데이터셋을 다운로드하고 압축을 해제합니다.
    !kaggle datasets download -d zynicide/wine-reviews
    !unzip -q wine-reviews.zip
    # model 디렉토리를 만듭니다.
    !mkdir models

Saving winemag-data-130k-v2.json to winemag-data-130k-v2.json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open wine-reviews.zip, wine-reviews.zip.zip or wine-reviews.zip.ZIP.


In [5]:
# 전체 데이터셋 로드
with open("./winemag-data-130k-v2.json") as json_data:
    wine_data = json.load(json_data)

In [6]:
wine_data[10]

{'points': '87',
 'title': 'Kirkland Signature 2011 Mountain Cuvée Cabernet Sauvignon (Napa Valley)',
 'description': 'Soft, supple plum envelopes an oaky structure in this Cabernet, supported by 15% Merlot. Coffee and chocolate complete the picture, finishing strong at the end, resulting in a value-priced wine of attractive flavor and immediate accessibility.',
 'taster_name': 'Virginie Boone',
 'taster_twitter_handle': '@vboone',
 'price': 19,
 'designation': 'Mountain Cuvée',
 'variety': 'Cabernet Sauvignon',
 'region_1': 'Napa Valley',
 'region_2': 'Napa',
 'province': 'California',
 'country': 'US',
 'winery': 'Kirkland Signature'}

In [7]:
# 데이터셋 필터링
filtered_data = [
    "wine review : "
    + x["country"]
    + " : "
    + x["province"]
    + " : "
    + x["variety"]
    + " : "
    + x["description"]
    for x in wine_data
    if x["country"] is not None
    and x["province"] is not None
    and x["variety"] is not None
    and x["description"] is not None
]

In [8]:
# 레시피 개수 카운트
n_wines = len(filtered_data)
print(f"{n_wines}개 레시피 로드")

129907개 레시피 로드


In [9]:
example = filtered_data[25]
print(example)

wine review : US : California : Pinot Noir : Oak and earth intermingle around robust aromas of wet forest floor in this vineyard-designated Pinot that hails from a high-elevation site. Small in production, it offers intense, full-bodied raspberry and blackberry steeped in smoky spice and smooth texture.


## **2. 데이터 토큰화** <a name="tokenize"></a>

In [10]:
# 구두점 앞에 공백을 채워서 별도의 '단어'로 취급합니다.
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [11]:
# 레시피 예시 표시
example_data = text_data[25]
example_data

'wine review : US : California : Pinot Noir : Oak and earth intermingle around robust aromas of wet forest floor in this vineyard - designated Pinot that hails from a high - elevation site . Small in production , it offers intense , full - bodied raspberry and blackberry steeped in smoky spice and smooth texture . '

In [12]:
# 텐서플로 데이터셋으로 변환하기
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [13]:
# TextVectorization 층 만들기
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [14]:
# 훈련 세트에 이 층을 적용합니다.
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [15]:
# 토큰:단어 매핑 표시
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: :
3: ,
4: .
5: and
6: the
7: wine
8: a
9: of


In [16]:
# 동일한 샘플을 정수로 변환하여 표시
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[   7   10    2   20    2   29    2   43   62    2   55    5  243 4145
  453  634   26    9  497  499  667   17   12  142   14 2214   43   25
 2484   32    8  223   14 2213  948    4  594   17  987    3   15   75
  237    3   64   14   82   97    5   74 2633   17  198   49    5  125
   77    4    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]


## **3. 훈련 세트 생성** <a name="create"></a>

In [17]:
# 레시피 텍스트와 한 단어만큼 이동된 동일한 텍스트로 훈련 세트를 만듭니다.
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

In [18]:
example_input_output = train_ds.take(1).get_single_element()

In [19]:
# 입력 예시
example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([   7,   10,    2,   20,    2,  150,    2,   43,   62,    2,  197,
          3,  180,    5,  871,    3,   12,   13,  127,  485,   24,   70,
          5,   24,  146,    4,   79,   18,   21,    8, 1538,    9, 3614,
         51,   71,    3,   23,    6,  341,   19,    8,  125,    3,  767,
         14,  107,   31,    4,   12,  399, 1526,   19, 1189,   19, 1019,
         84,  407,  121,   91,    4,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

In [20]:
# 출력 예시(토큰 하나씩 이동)
example_input_output[1][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  10,    2,   20,    2,  150,    2,   43,   62,    2,  197,    3,
        180,    5,  871,    3,   12,   13,  127,  485,   24,   70,    5,
         24,  146,    4,   79,   18,   21,    8, 1538,    9, 3614,   51,
         71,    3,   23,    6,  341,   19,    8,  125,    3,  767,   14,
        107,   31,    4,   12,  399, 1526,   19, 1189,   19, 1019,   84,
        407,  121,   91,    4,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

## **4. 코잘 어텐션 마스킹 함수 만들기** <a name="causal"></a>

- **Transformer 배경 및 역사**
    - **초기 Transformer** (2017): "Attention Is All You Need" 논문의 원조 Transformer는 '번역'을 위한 Encoder-Decoder 구조였음
    - **Encoder** (BERT 계열): Encoder는 입력 문장(예: 영어)을 이해하는 역할이었음. 문장을 깊게 이해하려면 문맥을 앞뒤로 다 보는 것이 유리했기에 **양방향 어텐션**을 사용했음. (예: BERT)
    - **Decoder** (GPT 계열): Decoder는 출력 문장(예: 한국어)을 생성하는 역할이었음. 한 단어씩 순차적으로 생성해야 했기에, **미래의 정답을 가리는 'Causal Masking' (논문에서는 'Look-Ahead Mask')이** 필수였음.
    - **GPT의 탄생**: "BERT가 Encoder만 사용했다면, 우리는 Decoder만 사용해서 '생성'에 특화된 거대 모델을 만들어보자!"라는 아이디어로 탄생한 것이 GPT임. 따라서 GPT는 태생부터 '**Causal Masking**'을 핵심 구조로 가진다.

In [21]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

## **5. 트랜스포머 블록 층 만들기** <a name="transformer"></a>

In [22]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

## **6. 토큰 임베딩과 위치 인코딩 만들기** <a name="embedder"></a>

In [23]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

## **7. 트랜스포머 모델 만들기** <a name="transformer_decoder"></a>

In [24]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)

outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)

gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [25]:
gpt.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, None, 256)      │     2,580,480 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ [(None, None, 256),    │       658,688 │
│ (TransformerBlock)              │ (None, 2, None, None)] │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 10000)    │     2,570,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,809,168 (22.16 MB)

 Trainable params: 5,809,168 (22.16 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    gpt = models.load_model("./models/gpt", compile=True)

## **8. 트랜스포머 훈련하기** <a name="train"></a>

In [33]:
# TextGenerator 체크포인트 만들기
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\n생성된 텍스트:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("wine review", max_tokens=80, temperature=1.0)

In [35]:
# 모델 저장 체크포인트 만들기
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt.weights.h5",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# 시작 프롬프트 토큰화
text_generator = TextGenerator(vocab)

In [36]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/5
4059/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.6030
생성된 텍스트:
wine review : germany : mosel : dornfelder : perfumed violet , iris , rose petal and elegant black cherry notes give this medium - bodied wine a balanced profile to delicately composed minerals and savory spice flavors . it ' s silky tannins and a touch of mineral note frame the finish . 

4060/4060 ━━━━━━━━━━━━━━━━━━━━ 198s 46ms/step - loss: 2.6028
Epoch 2/5
4059/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.9830
생성된 텍스트:
wine review : us : washington : syrah : this new value - minded wine shows its elegant styling and a bit disjointed , it brings a bitter edge , peppery herbs right now , bringing an outstanding value . 

4060/4060 ━━━━━━━━━━━━━━━━━━━━ 116s 29ms/step - loss: 1.9830
Epoch 3/5
4060/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 1.8985
생성된 텍스트:
wine review : us : washington : bordeaux - style red blend : fermented in concrete tanks , this opens with broad , ripe black - berry fruit , coffee 

In [38]:
# 최종 모델 저장
gpt.save("./models/gpt.keras")

## **9. 트랜스포머를 사용하여 텍스트 생성**
- 앞에서 만든 신경망은 각 단어에 대해 샘플링이 가능한 확률을 출력함
- temperature : 샘플링 과정이 얼마나 결정적인지 나타내는 변수,0에 가까울 수록 결정적

In [39]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        # 문장에 있는 이전 단어에 대한 소프트맥스 분포 보여주기(상위 5개 top_k=5)
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):       # 문장에 있는 이전 단어에 대한 소프트맥스 분포
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

- 세 가지 다른 입력 프롬프트에 대해서 확률이 가장 높은 상위 다섯 개 토큰과
- 이전 단어에 대한 두 헤드의 평균 어텐션을 보여줌

In [45]:
info = text_generator.generate(
    "wine review : us", max_tokens=80, temperature=1.0
)


생성된 텍스트:
wine review : us : new york : riesling : wisps of smoke and bruised apple offset , freshly pressed pressed apples and pears are curious , along with cassis and green herbs on the nose . the palate is lightweight and extends through an easy finish . 



In [46]:
info = text_generator.generate(
    "wine review : us", max_tokens=80, temperature=1.0
)
print_probs(info, vocab)


생성된 텍스트:
wine review : us : oregon : pinot noir : a standout among the most successful single - vineyard pinot noir burgundy clones make this already smooth inexpensive , rounded , surprisingly drinkable is a crowd pleaser . all of the mixed , ripe cranberry and cherry fruit proves to be spicy and inviting , with a clean , mouthwatering finish . 



::   	99.98999786376953%
-:   	0.0%
grosso:   	0.0%
with:   	0.0%
and:   	0.0%
--------



california:   	63.41999816894531%
washington:   	14.979999542236328%
oregon:   	13.319999694824219%
new:   	4.940000057220459%
virginia:   	1.4500000476837158%
--------



::   	99.98999786376953%
-:   	0.0%
grosso:   	0.0%
hills:   	0.0%
other:   	0.0%
--------



pinot:   	65.30000305175781%
chardonnay:   	9.579999923706055%
riesling:   	5.900000095367432%
cabernet:   	2.0199999809265137%
syrah:   	2.009999990463257%
--------



...

,:   	49.06999969482422%
finish:   	25.15999984741211%
and:   	1.7599999904632568%
streak:   	1.4700000286102295%
line:   	1.4199999570846558%
--------



polished:   	8.819999694824219%
refreshing:   	6.5%
supple:   	4.28000020980835%
fresh:   	3.9800000190734863%
mouthwatering:   	3.240000009536743%
--------



finish:   	93.94000244140625%
mouthfeel:   	0.6100000143051147%
aftertaste:   	0.4000000059604645%
feel:   	0.38999998569488525%
texture:   	0.3100000023841858%
--------



.:   	97.43000030517578%
,:   	0.7699999809265137%
that:   	0.46000000834465027%
with:   	0.17000000178813934%
and:   	0.14000000059604645%
--------



:   	85.98999786376953%
drink:   	2.6700000762939453%
it:   	1.2100000381469727%
this:   	1.0499999523162842%
a:   	0.7099999785423279%
--------



In [41]:
info = text_generator.generate(
    "wine review : italy", max_tokens=80, temperature=0.5
)


생성된 텍스트:
wine review : italy : tuscany : sangiovese : underbrush , leather , moist soil and tobacco aromas lead the nose . the palate delivers tart red cherry , crushed raspberry , white pepper and a hint of pipe tobacco alongside firm tannins . 



In [47]:
info = text_generator.generate(
    "wine review : italy", max_tokens=80, temperature=0.5
)
print_probs(info, vocab)


생성된 텍스트:
wine review : italy : piedmont : nebbiolo : aromas of perfumed berry , pressed rose and cake spice lead the nose . the straightforward palate offers raspberry jam , vanilla and a hint of cake spice alongside a hint of dried sage . it ' s already accessible , with bright acidity and firm , fine - grained tannins . drink 2021–2028 . 



::   	100.0%
grosso:   	0.0%
-:   	0.0%
with:   	0.0%
other:   	0.0%
--------



piedmont:   	65.77999877929688%
tuscany:   	25.3799991607666%
northeastern:   	3.7100000381469727%
veneto:   	3.319999933242798%
southern:   	0.8799999952316284%
--------



::   	100.0%
-:   	0.0%
grosso:   	0.0%
other:   	0.0%
with:   	0.0%
--------



nebbiolo:   	98.51000213623047%
barbera:   	1.3200000524520874%
moscato:   	0.05999999865889549%
dolcetto:   	0.05000000074505806%
arneis:   	0.03999999910593033%
--------



...

.:   	96.69999694824219%
that:   	1.8799999952316284%
and:   	1.409999966621399%
,:   	0.009999999776482582%
for:   	0.0%
--------



:   	65.87000274658203%
drink:   	33.150001525878906%
enjoy:   	0.6899999976158142%
it:   	0.14000000059604645%
a:   	0.10000000149011612%
--------



through:   	59.029998779296875%
2021–2028:   	34.25%
2018–2028:   	1.2599999904632568%
2018–2026:   	0.8100000023841858%
2017–2022:   	0.6499999761581421%
--------



.:   	100.0%
,:   	0.0%
through:   	0.0%
and:   	0.0%
:   	0.0%
--------



:   	100.0%
.:   	0.0%
enjoy:   	0.0%
this:   	0.0%
worldwide:   	0.0%
--------

